## Library Imports

In [1]:
# Import dataset class and accuracy functions
from utils.Dataloader import PricingWizardDataset
from utils.RegressionEvaluation import (regression_accuracy,  threshold_accuracy)
from utils.DataTransformation import (base_regression_pipeline, ridge_regression_pipeline)
from models.regression_neural_network import RegressionNN
from utils.helpers import load_model, drop_helpers 
from utils.NeuralNetHelpers import test, set_device

# Model loading imports
import joblib
import torch

# Import other libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
# Load Data
data = PricingWizardDataset()

Dataset Loaded: post_preprocessing_without_dummies
	Number of Rows: 283055
	Number of Columns: 22
	Outlier Removal: True
	Train Size: 0.8
	Test Size: 0.2
	Random State: 42


In [3]:
# Current working directory
cwd = os.getcwd()

# model directory
model_dir = os.path.join(cwd, 'models', 'pickled_models')

In [4]:
# Files in model directory
print(os.listdir(model_dir))

['.DS_Store', 'prediction_svr_linear.pkl', 'regularized_regression.pkl', 'prediction_random_forest.pkl.zip', 'prediction_random_forest.pkl', 'base_regression.pkl', 'regression_neural_net.pt']


#### Base Regression Model

In [5]:
# Reset dataset as standard
data.reset_dataset()

In [6]:
# Perform preprocessing
data.apply_function(base_regression_pipeline)


# Load model
base_regression = load_model(f'{model_dir}/base_regression.pkl')

# Split
_, X_test, _, y_test = data.stratify_train_test_split(val_size=0,
                                                      y_column='log_listing_price')

Model loaded successfully from /Users/rasmuskrebs/Documents/School/semester_3/data_mining/pricing_wizards/models/pickled_models/base_regression.pkl
Dependent variable distribution is equal across all subsets


In [7]:
# Predict
regression_prediction = base_regression.predict(drop_helpers(X_test))

# Calculate accuracy
print('Log Scale Accuracy')
regression_accuracy(regression_prediction, y_test)
threshold_accuracy(regression_prediction, y_test, p=0.1)

print('\nOriginal Scale Accuracy')
regression_accuracy(regression_prediction, y_test, return_metrics=False, scale_up=True)
threshold_accuracy(regression_prediction, y_test, p=0.1, scale_up=True)

Log Scale Accuracy
R2 Score: 0.5454074633046996
MSE: 0.43798999970075375
MAE 0.5134088438732489
RMSE 0.6618081290682019
Threshold Accuracy 0.6385154828566886

Original Scale Accuracy
R2 Score: 0.3491713283801632
MSE: 193549.2874597735
MAE 211.4213411969003
RMSE 439.9423683390513
Threshold Accuracy 0.12686580346575754


#### Regularized Regression Model

In [9]:
# Reset dataset as standard
data.reset_dataset()

# Perform preprocessing
data.apply_function(ridge_regression_pipeline)

# Load model
ridge_regression = load_model(f'{model_dir}/regularized_regression.pkl')

# Split
_, X_test, _, y_test = data.stratify_train_test_split(val_size=0,
                                                      y_column='log_listing_price')

Model loaded successfully from /Users/rasmuskrebs/Documents/School/semester_3/data_mining/pricing_wizards/models/pickled_models/regularized_regression.pkl
Dependent variable distribution is equal across all subsets


In [10]:
# Predict
ridge_regression_prediction = ridge_regression.predict(drop_helpers(X_test))

# Calculate accuracy
print('Log Scale Accuracy')
regression_accuracy(ridge_regression_prediction, y_test)
threshold_accuracy(ridge_regression_prediction, y_test, p=0.1)

print('\nOriginal Scale Accuracy')
regression_accuracy(ridge_regression_prediction, y_test, scale_up=True)
threshold_accuracy(ridge_regression_prediction, y_test, p=0.1, scale_up=True)


Log Scale Accuracy
R2 Score: 0.6060281080749564
MSE: 0.37958332989970334
MAE 0.4704520986562844
RMSE 0.6161033435225809
Threshold Accuracy 0.6783134019890127

Original Scale Accuracy
R2 Score: 0.4562210332121559
MSE: 161713.8828494586
MAE 191.055882600751
RMSE 402.1366469863927
Threshold Accuracy 0.15161364399145041


### Support Vector Machines (SVM)

In [13]:
# Reset dataset as standard
data.reset_dataset()

In [14]:
# Perform preprocessing
data.apply_function(base_regression_pipeline)

# Load model
linear_svr = load_model(f'{model_dir}/prediction_svr_linear.pkl')

# Split
_, X_test, _, y_test = data.stratify_train_test_split(val_size=0,
                                                      y_column='log_listing_price')

/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVR from version 1.3.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Model loaded successfully from /Users/rasmuskrebs/Documents/School/semester_3/data_mining/pricing_wizards/models/pickled_models/prediction_svr_linear.pkl
Dependent variable distribution is equal across all subsets


In [15]:
# StandardScaler
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

In [16]:
# Predict
svr_prediction = linear_svr.predict(X_test_scaled)

# Calculate accuracy
print('Log Scale Accuracy')
regression_accuracy(svr_prediction, y_test)
threshold_accuracy(svr_prediction, y_test, p=0.1)

print('\nOriginal Scale Accuracy')
regression_accuracy(svr_prediction, y_test, return_metrics=False, scale_up=True)
threshold_accuracy(svr_prediction, y_test, p=0.1, scale_up=True)

Log Scale Accuracy
R2 Score: 0.545466400362267
MSE: 0.4379332151305231
MAE 0.5129098463702417
RMSE 0.6617652265951446
Threshold Accuracy 0.6390277507904825

Original Scale Accuracy
R2 Score: 0.35351415798828056
MSE: 192257.77770787783
MAE 211.55830725512112
RMSE 438.47209456005044
Threshold Accuracy 0.12861458020526045


### Random Forest

In [34]:
# Reset dataset as standard
data.reset_dataset()

# Perform preprocessing
data.apply_function(base_regression_pipeline)

# Split
X_train, X_test, y_train, y_test = data.stratify_train_test_split(val_size=0,
                                                      y_column='log_listing_price')

Dependent variable distribution is equal across all subsets


In [35]:
# Extract model
with zipfile.ZipFile(f'{model_dir}/prediction_random_forest.pkl.zip', 'r') as zip_ref:
    # Extract all contents to the specified directory
    zip_ref.extractall(model_dir)

In [36]:
# Load model
random_forest = load_model(f'{model_dir}/prediction_random_forest.pkl')

Model loaded successfully from /Users/rasmuskrebs/Documents/School/semester_3/data_mining/pricing_wizards/models/pickled_models/prediction_random_forest.pkl


In [37]:
# Predict
rf_prediction = random_forest.predict(X_test.values)

# Calculate accuracy
print('Log Scale Accuracy')
regression_accuracy(rf_prediction, y_test)
threshold_accuracy(rf_prediction, y_test, p=0.1)

print('\nOriginal Scale Accuracy')
regression_accuracy(rf_prediction, y_test, return_metrics=False, scale_up=True)
threshold_accuracy(rf_prediction, y_test, p=0.1, scale_up=True)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Log Scale Accuracy
R2 Score: 0.6475837626045777
MSE: 0.339545362608584
MAE 0.4353815559570817
RMSE 0.5827052107271601
Threshold Accuracy 0.7054282736570631

Original Scale Accuracy
R2 Score: 0.5405418018555301
MSE: 136637.8138306086
MAE 170.49494265737152
RMSE 369.6455245645598
Threshold Accuracy 0.18325060500609422


### Neural Net

In [17]:
# Reset dataset, used during modelling and overwrites any previous changes
data.reset_dataset()

# Apply ridge regression pipeline
data.apply_function(ridge_regression_pipeline)

# Standard Scaling
scaler = MinMaxScaler()
X = scaler.fit_transform(drop_helpers(data.df))

# Assigning X to data.df
data.df[drop_helpers(data.df).columns] = X

In [18]:
# Splitting dataset
X_train, X_test, y_train, y_test  = data.stratify_train_test_split(val_size=0, return_splits=True, y_column='log_listing_price')

# Converting to PyTorch tensors
X_test_tensor = torch.tensor(drop_helpers(X_test).to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

Dependent variable distribution is equal across all subsets


In [19]:
# Load model
network = RegressionNN(input_size=X_test_tensor.shape[1])

# Load weights
network.load_state_dict(torch.load(f'{model_dir}/regression_neural_net.pt'))

<All keys matched successfully>

In [20]:
# Determine device
device = set_device()

nn_prediction = test(network, X_test_tensor, device)

In [21]:
# Calculate accuracy
print('Log Scale Accuracy')
regression_accuracy(nn_prediction, y_test)
threshold_accuracy(nn_prediction, y_test, p=0.1)

print('\nOriginal Scale Accuracy')
regression_accuracy(nn_prediction, y_test, scale_up=True)
threshold_accuracy(nn_prediction, y_test, p=0.1, scale_up=True)


Log Scale Accuracy
R2 Score: 0.6408822819519124
MSE: 0.3460020931356468
MAE 0.4392821771925936
RMSE 0.588219426010096
Threshold Accuracy 0.7047216971966579

Original Scale Accuracy
R2 Score: 0.5474373442587677
MSE: 134587.15537471496
MAE 173.7124975176537
RMSE 366.8612208652135
Threshold Accuracy 0.1768207592164067
